# DiploDatos 2019 - Población Penitenciaria en Argentina




> http://diplodatos.famaf.unc.edu.ar/poblacion-penitenciaria-en-argentina-2002-a-2017/




## Práctico Introducción al Aprendizaje Automático

En este práctico utilizaremos algunos algoritmos básicos de aprendizaje automático para intentar encontrar patrones en los datos, y evaluaremos los resultados usando métricas estándares para cada tipo de problema. 

El objetivo del práctico es que pensemos en las distintas formas que puede tomar el modelado de un problema con datos, y en el trabajo de selección de features que debe realizarse para poder abordarlo.

Para ello intentaremos responder la siguiente pregunta:

¿Es posible predecir la duración de la condena que recibirá una persona en base a datos como los que se encuentran presentes en este dataset?

Tener en cuenta los siguientes lineamientos:  

- Utilicen los registros del año 2017 
- Deben decidir a partir de qué valor de una métrica determinada consideran aceptable el resultado. Esto lo tienen que hacer antes de empezar a entrenar un modelo.
- Sólo pueden utilizar las variables que corresponden a información generada antes que la persona ingrese a la institución penal.
- Deberán trabajar con el problema de dos maneras diferentes: como un problema de regresión, y como un problema de clasificación.
- A la hora de hacer selección de variables, tengan en cuenta que los modelos de Machine Learning solo reciben inputs numéricos
- Es importante que almacenen los resultados de alguna manera para poder comparar los distintos modelos. Recomendación: crear una función para almacenar resultados en un dataframe de pandas e imprimirlos en pantalla. 

#### Inicialización y carga de datos

In [24]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.linear_model import LinearRegression,LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.model_selection import GridSearchCV

np.random.seed(0)

pd.set_option('display.max_columns', 60)

In [25]:
# Ejemplo, modificar acorde a la ubicación donde tengan su set de datos
data_raw = pd.read_csv('../datasets/sneep_2017_diplodatos.csv', index_col=0)

In [26]:
data_raw.sample(5)

,anio_censo,provincia_descripcion,establecimiento_descripcion,edad,genero_descripcion,nacionalidad_descripcion,estado_civil_descripcion,nivel_instruccion_descripcion,ultima_situacion_laboral_descripcion,capacitacion_laboral_al_ingresar_descripcion,ultimo_lugar_residencia_descripcion,ultima_provincia_residencia_descripcion,jurisdiccion_descripcion,situacion_legal_descripcion,fecha_detencion,fecha_condenado,establecimiento_procedencia_descripcion,delito1_descripcion,delito2_descripcion,delito3_descripcion,delito4_descripcion,delito5_descripcion,horas_trabajo_remunerado_descripcion,participacion_programa_laboral,participacion_programa_educativo_descripcion,participacion_actividades_deportivas,recibio_atencion_medica_ult_anio_descripcion,recibio_visitas_ultimo_anio,participo_alteracion_orden_ult_anio_descripcion,tipo_infraccion_disciplinaria_descripcion,sancion_aplicada_descripcion,calificacion_conducta_descripcion,tentativa_fugas_evasiones_descripcion,tentativa_suicidio,fue_lesionado_descripcion,duracion_condena_anios,duracion_condena_meses,duracion_condena_rango,tipo_condena,tiene_medidas_seguridad,es_reincidente_descripcion,tiene_periodo_progresividad_descripcion,reparticion_descripcion,tuvo_salidas_transitorias_descripcion,incorporado_reg_semi_libertad_descripcion,participa_programa_pre_libertad,participa_programa_prision_discontinua_descripcion,participa_programa_semi_detencion_descripcion,tuvo_reduccion_pena_descripcion,mujer_tiene_hijos_intramuro
53495,2017,Chaco,SPF - U. 7 PRISION REGIONAL DEL NORTE,45.0,Masculino,Argentina,Soltero,Primario completo,Desocupado,No tiene ni oficio ni profesión,Urbano,Ciudad de Buenos Aires,FEDERAL,Condenado,1994-07-15,2012-04-12,Deriv. de otro establecimiento,Homicidios dolosos,NaN,NaN,NaN,NaN,Hasta 40 hs. Semanales,1.0,Si - educación formal - SECUNDARIA (Polimodal),1.0,SI - HIPERTENSIÓN,1.0,No participó de ninguna alteración,No cometió Infracción disciplinaria,NaN,Ejemplar,No,0.0,No fue lesionado,30.0,6.0,7,0.0,3.0,Reincidente,Período de tratamiento,Federal,No le fueron otorgadas,No está incorporado al régimen de semilibertad,1.0,No está incorporado,No tiene semidetención,No tuvo reducción de pena,NaN
75586,2017,San Luis,COMPLEJO PENITENCIARIO PROVINCIAL - UNIDAD 2 ...,25.0,Masculino,Argentina,Soltero,Secundario incompleto,Trabajador de tiempo parcial,Tiene algún oficio,Urbano,Mendoza,FEDERAL,Procesado,2017-04-19,NaN,Deriv. de una instit. Policial,Infracción ley n° 23.737 (estupefacientes),NaN,NaN,NaN,NaN,No tiene trabajo remunerado,0.0,No participa de ningún programa educativo,1.0,Si - Recibió Asistencia Médica,1.0,"SI - Alteración del orden SIN heridos, ni rehenes",Faltas graves,Otra Sanción,Sin calificación,No,0.0,No fue lesionado,0.0,0.0,0,NaN,0.0,NaN,NaN,Provincial,NaN,NaN,0.0,NaN,NaN,NaN,NaN
181,2017,Buenos Aires,UNIDAD 13 - JUNIN,42.0,Masculino,Argentina,Soltero,Secundario completo,Trabajador de tiempo parcial,Tiene alguna profesión,Urbano,Buenos Aires,PROVINCIAL - Buenos Aires,Procesado,2016-04-26,NaN,Deriv. de otro establecimiento,Violaciones,NaN,NaN,NaN,NaN,No tiene trabajo remunerado,0.0,No participa de ningún programa educativo,1.0,SI - ASMA O EPOC,1.0,No participó de ninguna alteración,No cometió Infracción disciplinaria,NaN,NaN,No,0.0,No fue lesionado,0.0,0.0,0,NaN,0.0,NaN,NaN,Provincial,NaN,NaN,0.0,NaN,NaN,NaN,NaN
3530,2017,Buenos Aires,UNIDAD 33 - LOS HORNOS (MUJERES),42.0,Femenino,Argentina,Soltero,Primario completo,Desocupado,No tiene ni oficio ni profesión,Urbano,Buenos Aires,PROVINCIAL - Buenos Aires,Condenado,2015-08-01,2013-02-27,Ingreso directo,Otros delitos contra la propiedad,NaN,NaN,NaN,NaN,No tiene trabajo remunerado,0.0,Si - educación formal - SECUNDARIA (Polimodal),0.0,SI - HIV,1.0,No participó de ninguna alteración,Faltas media,Exclus. de activ. recreat/dep. hasta 10 días,Ejemplar,No,0.0,No fue lesionado,5.0,0.0,2,0.0,1.0,Primario,No se aplica ninguna,Provincial,No le fueron otorgadas,No está incorporado al régimen de semilibertad,1.0,No está incorporado,No tie

### Regresión

1- Filtrar los registros, conservar sólo aquellos que tengan condena

In [27]:
## Campos que estan relacionados con la condena
for col in data_raw.columns:
    if 'condena' in col:
        print(col)

fecha_condenado
duracion_condena_anios
duracion_condena_meses
duracion_condena_rango
tipo_condena


In [28]:
data_raw.columns

Index(['anio_censo', 'provincia_descripcion', 'establecimiento_descripcion',
       'edad', 'genero_descripcion', 'nacionalidad_descripcion',
       'estado_civil_descripcion', 'nivel_instruccion_descripcion',
       'ultima_situacion_laboral_descripcion',
       'capacitacion_laboral_al_ingresar_descripcion',
       'ultimo_lugar_residencia_descripcion',
       'ultima_provincia_residencia_descripcion', 'jurisdiccion_descripcion',
       'situacion_legal_descripcion', 'fecha_detencion', 'fecha_condenado',
       'establecimiento_procedencia_descripcion', 'delito1_descripcion',
       'delito2_descripcion', 'delito3_descripcion', 'delito4_descripcion',
       'delito5_descripcion', 'horas_trabajo_remunerado_descripcion',
       'participacion_programa_laboral',
       'participacion_programa_educativo_descripcion',
       'participacion_actividades_deportivas',
       'recibio_atencion_medica_ult_anio_descripcion',
       'recibio_visitas_ultimo_anio',
       'participo_alteracion_orde

In [32]:
def preprocess(data):
    return data

def remove_descripcion(data):
    rename_dict = {}
    for col in data.columns:
        if 'descripcion' in col:
            rename_dict[col] = '_'.join(col.split('_')[:-1])
    data = data.rename(columns=rename_dict, inplace=False)
    return data

def filtrar_condena(data):
    mask = data.tipo_condena.notnull()
    #print(mask)
    filtered_data = data[mask]
    filtered_data.tipo_condena.astype('int')
    return filtered_data

def eliminar_columnas(data):
    drop_cols = [
        'tuvo_reduccion_pena',
        'participa_programa_semi_detencion',
        'participa_programa_prision_discontinua',
        'tiene_periodo_progresividad',
        'tuvo_salidas_transitorias',
        'incorporado_reg_semi_libertad',
        'participa_programa_pre_libertad',
        'situacion_legal',
        'participacion_programa_laboral',
        'participacion_programa_educativo',
        'participacion_actividades_deportivas',
        'recibio_atencion_medica_ult_anio',
        'recibio_visitas_ultimo_anio',
        'participo_alteracion_orden_ult_anio',
        'tipo_infraccion_disciplinaria',
        'sancion_aplicada',
        'calificacion_conducta',
        'tentativa_fugas_evasiones',
        'tentativa_suicidio',
        'horas_trabajo_remunerado',
        'fue_lesionado',
        'anio_censo'
    ]
    data = data.drop(drop_cols, axis=1)
    return data

def encode_data(data):
    dummy_cols = [
        'provincia'
    ]
    data = pd.get_dummies(data, columns=dummy_cols)
    return data

filtered_data = remove_descripcion(data_raw)
filtered_data = filtrar_condena(data)
filtered_data = eliminar_columnas(filtered_data)
filtered_data = encode_data(filtered_data)
print(filtered_data.shape)
filtered_data.head()

(43457, 51)


,establecimiento,edad,genero,nacionalidad,estado_civil,nivel_instruccion,ultima_situacion_laboral,capacitacion_laboral_al_ingresar,ultimo_lugar_residencia,ultima_provincia_residencia,jurisdiccion,fecha_detencion,fecha_condenado,establecimiento_procedencia,delito1,delito2,delito3,delito4,delito5,duracion_condena_anios,duracion_condena_meses,duracion_condena_rango,tipo_condena,tiene_medidas_seguridad,es_reincidente,reparticion,mujer_tiene_hijos_intramuro,provincia_Buenos Aires,provincia_Catamarca,provincia_Chaco,provincia_Chubut,provincia_Ciudad de Buenos Aires,provincia_Corrientes,provincia_Córdoba,provincia_Entre Rios,provincia_Formosa,provincia_Jujuy,provincia_La Pampa,provincia_La Rioja,provincia_Mendoza,provincia_Misiones,provincia_Neuquén,provincia_Río Negro,provincia_Salta,provincia_San Juan,provincia_San Luis,provincia_Santa Cruz,provincia_Santa Fe,provincia_Santiago del Estero,provincia_Tierra del Fuego,provincia_Tucumán
0,UNIDAD 13 - JUNIN,28.0,Masculino,Argentina,Soltero,Primario incompleto,Desocupado,No tiene ni oficio ni profesión,Urbano,Buenos Aires,PROVINCIAL - Buenos Aires,2017-02-23,2017-08-11,Deriv. de una instit. Policial,Otros delitos contra las personas,Amenazas,NaN,NaN,NaN,2.0,2.0,1,0.0,1.0,Primario,Provincial,NaN,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
282,UNIDAD 13 - JUNIN,37.0,Masculino,Argentina,Casado,Ninguno,Desocupado,No tiene ni oficio ni profesión,Urbano,Buenos Aires,PROVINCIAL - Buenos Aires,2017-02-03,2017-12-03,Deriv. de otro establecimiento,Delitos contra la seguridad pública,NaN,NaN,NaN,NaN,3.0,10.0,2,0.0,1.0,Primario,Provincial,NaN,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
283,UNIDAD 13 - JUNIN,22.0,Masculino,Argentina,Casado,Primario completo,Trabajador de tiempo parcial,Tiene alguna profesión,Urbano,Buenos Aires,PROVINCIAL - Buenos Aires,2013-03-31,2013-05-14,Deriv. de otro establecimiento,Robo y/o tentativa de robo,NaN,NaN,NaN,NaN,5.0,0.0,2,0.0,1.0,Primario,Provincial,NaN,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
284,UNIDAD 13 - JUNIN,23.0,Masculino,Argentina,Viudo,Primario incompleto,Desocupado,Tiene alguna profesión,Urbano,Buenos Aires,PROVINCIAL - Buenos Aires,2015-02-02,2016-10-11,Deriv. de otro establecimiento,Homicidios dolosos (tent.),NaN,NaN,NaN,NaN,8.0,7.0,3,0.0,1.0,Primario,Provincial,NaN,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
285,UNIDAD 13 - JUNIN,23.0,Masculino,Paraguaya,Casado,Primario completo,Desocupado,Tiene alguna profesión,Urbano,Buenos Aires,PROVINCIAL - Buenos Aires,2016-02-24,2017-05-14,Deriv. de otro establecimiento,Infracción ley n° 23.737 (estupefacientes),NaN,NaN,NaN,NaN,4.0,3.0,2,0.0,1.0,Primario,Provincial,NaN,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


2- La variable a predecir que utilizaremos será la condena en meses, generar la feature correspondiente, anexarla al dataset

In [33]:
filtered_data.dropna(subset=['duracion_condena_anios', 'duracion_condena_meses'],inplace=True)
filtered_data['duracion_condena'] = 12*filtered_data['duracion_condena_anios'] + \
                                       filtered_data['duracion_condena_meses']

3- Generar dataframe con las variables de tipo "delito_xx" y la feature generada en el paso anterior. Dividir los datos en conjunto de training y test.

In [39]:
delito_columns = ['delito{:d}'.format(i) for i in range(1, 6)]
columns = delito_columns + ['duracion_condena']
data = filtered_data[columns]
df1 = pd.DataFrame(data, columns=columns)
df1

df_delitos = df1.copy()
df_delitos[delito_columns] = df_delitos[delito_columns].astype('category')
cat_columns = df_delitos.select_dtypes(['category']).columns
df_delitos[cat_columns] = df_delitos[cat_columns].apply(lambda x: x.cat.codes)
df_delitos.head()

X = df_delitos[delito_columns]
y = df_delitos['duracion_condena']

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)

4- Generar un modelo baseline, esto es, un modelo "tonto" que devuelva siempre el promedio de las condenas para el crimen cometido (usaremos solo delito1_id). Calcular 3 métricas de evaluación de regresiones para este modelo. 

In [40]:
X1_train, X1_test = X_train['delito1'], X_test['delito1']

In [41]:
def predict_condena_naive(X_train, X_test, y_train):
    
    y_pred = np.zeros(X_test.shape[0])
    for i, x in enumerate(X_test):
        mask = X_train == x
        y_pred[i] = y_train[mask].mean()
        
    return y_pred

def compute_metrics(y_true, y_pred):
    mean_sq_err = mean_squared_error(y_true, y_pred)
    mean_abs_err = mean_absolute_error(y_true, y_pred)
    r2 = r2_score(y_true, y_pred)
    return np.sqrt(mean_sq_err), mean_abs_err, r2

y_pred = predict_condena_naive(X1_train, X1_test, y_train)

compute_metrics(y_test, y_pred)

(60.1461377260645, 42.46140345131342, 0.1361488399061621)

5- Entrenar una regresión lineal y un árbol de decisión usando solo las variables de tipo "delitoxxx". Evaluar métricas para el conjunto de training y para test.

In [42]:
## Linear regressor
reg = LinearRegression().fit(X_train, y_train)
y_pred = reg.predict(X_test)
print(compute_metrics(y_test, y_pred))

## Tree regressor
reg = DecisionTreeRegressor().fit(X_train, y_train)
y_pred = reg.predict(X_test)
print(compute_metrics(y_test, y_pred))

(64.46419835275495, 48.83360378391409, 0.007659755342946273)
(61.3962442664195, 42.4887442839945, 0.09986625646392744)


6- Seleccionar 3 variables adicionales para entrenar los modelos. Generar nuevamente conjuntos de training y test. Entrenar y evaluar los modelos de regresión lineal y árbol de decisión.

In [43]:
### Selecciono variables: edad, nacionalidad y estado civil
variables  = ['edad', 'nacionalidad', 'estado_civil']
feature_columns = variables + delito_columns
columns = feature_columns + ['duracion_condena']
data = filtered_data[columns]
df2 = pd.DataFrame(data, columns=columns)
df2

df_delitos2 = df2.copy()
df_delitos2[feature_columns] = df_delitos2[feature_columns].astype('category')
cat_columns = df_delitos2.select_dtypes(['category']).columns
df_delitos2[cat_columns] = df_delitos2[cat_columns].apply(lambda x: x.cat.codes)

X = df_delitos2[feature_columns]
y = df_delitos2['duracion_condena']

X_train, X_test, y_train, y_test = train_test_split(X, y)

## Linear regressor
print('Linear regressor')
reg = LinearRegression().fit(X_train, y_train)
y_pred = reg.predict(X_train)
print('Train', compute_metrics(y_train, y_pred))
y_pred = reg.predict(X_test)
print('Test', compute_metrics(y_test, y_pred))

## Tree regressor
print('Desicion tree')
reg = DecisionTreeRegressor().fit(X_train, y_train)
y_pred = reg.predict(X_train)
print('Train', compute_metrics(y_train, y_pred))
y_pred = reg.predict(X_test)
print('Test', compute_metrics(y_test, y_pred))

Linear regressor
Train (63.719330992784506, 47.46429554721406, 0.04811855695886991)
Test (63.7493351310075, 47.0398912824484, 0.04129268177052181)
Desicion tree
Train (48.37152988709433, 30.681177818216476, 0.45144510807752714)
Test (68.92972520241122, 46.431456999892326, -0.1208508291337771)


### One-hot encoding de delitos

In [44]:
import math
def lista_delitos(df):
    delito_columns = ['delito{:d}'.format(i) for i in range(1, 6)]
    df_delito = df[delito_columns]
    
    delitos = pd.unique(df_delito.values.ravel('K')).tolist()
    delitos = list(filter(lambda x: isinstance(x, str), delitos))
    
    n_delitos = len(delitos)
    n_rows = df.shape[0]
    matrix = np.zeros((n_rows, n_delitos), dtype='int')
    delito_dict = {delito: i for i, delito in enumerate(delitos)} 
    
    for i, (index, row) in enumerate(df_delito.iterrows()):
        for delito in row:
            if not isinstance(delito, str):
                continue
            j = delito_dict[delito]
            matrix[i,j] = 1
        
    out_columns = delitos + ['duracion_condena']
    out_df = pd.DataFrame(data=matrix, columns=delitos)
    out_df.index = df.index
    out_df['duracion_condena'] = df['duracion_condena']
    return out_df

new_df = lista_delitos(filtered_data)
new_df.head()

,Otros delitos contra las personas,Delitos contra la seguridad pública,Robo y/o tentativa de robo,Homicidios dolosos (tent.),Infracción ley n° 23.737 (estupefacientes),Amenazas,Violaciones,Homicidios dolosos,Hurto y/o tentativa de hurto,Otros delitos contra la integridad sexual,Privación ilegítima de la libertad,Delitos c/el orden público,Delitos contra el honor,Otros delitos contra la propiedad,Homicidios Culposos,Lesiones Culposas,Lesiones Dolosas,Otros delitos contra la libertad,Delitos contra los poderes publicos,Del. contra la lib. comet. por func. público,Delitos c/ la fe pública,Delitos c/ la administracion pública,Delitos previstos en leyes especiales,Infraccion ley n° 13.944 incumplimiento de deberes,Delitos c/ el estado civil,Contravenciones,Infraccion ley n° 24.769 penal tributaria,Delitos contra la seguridad de la nación,duracion_condena
0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,26.0
282,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,46.0
283,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,60.0
284,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,103.0
285,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,51.0


In [45]:
X = new_df.iloc[:,:-1]
y = new_df['duracion_condena']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

## Linear regressor
print('Linear regressor')
reg = LinearRegression().fit(X_train, y_train)
y_pred = reg.predict(X_train)
print('Train', compute_metrics(y_train, y_pred))
y_pred = reg.predict(X_test)
print('Test', compute_metrics(y_test, y_pred))

## Tree regressor
print('Desicion tree')
reg = DecisionTreeRegressor().fit(X_train, y_train)
y_pred = reg.predict(X_train)
print('Train', compute_metrics(y_train, y_pred))
y_pred = reg.predict(X_test)
print('Test', compute_metrics(y_test, y_pred))

Linear regressor
Train (60.36748337733949, 42.33211509638702, 0.14269930569152012)
Test (60.652029598776195, 42.39027568975446, 0.14266026642681895)
Desicion tree
Train (58.46475350231653, 40.476339002018726, 0.19589034087821433)
Test (60.727562345892515, 42.03740426193789, 0.1405235680084821)


### Clasificación

1- Filtrar los registros, conservar sólo aquellos que tengan condena

2- Para poder La variable a predecir debe dividir las condenas en rangos, por ejemplo, las condenas de 1 a 3 años dentro del rango 1, de 3 a 5 en el 2, etc. Analizar el dataset para determinar qué utilizar.

In [46]:
delito_columns = ['delito{:d}'.format(i) for i in range(1, 6)]
columns = delito_columns + ['duracion_condena_rango']
data = filtered_data[columns]
df1 = pd.DataFrame(data, columns=columns)
df1

,delito1,delito2,delito3,delito4,delito5,duracion_condena_rango
0,Otros delitos contra las personas,Amenazas,NaN,NaN,NaN,1
282,Delitos contra la seguridad pública,NaN,NaN,NaN,NaN,2
283,Robo y/o tentativa de robo,NaN,NaN,NaN,NaN,2
284,Homicidios dolosos (tent.),NaN,NaN,NaN,NaN,3
285,Infracción ley n° 23.737 (estupefacientes),NaN,NaN,NaN,NaN,2
286,Robo y/o tentativa de robo,NaN,NaN,NaN,NaN,2
287,Robo y/o tentativa de robo,Amenazas,NaN,NaN,NaN,2
289,Amenazas,NaN,NaN,NaN,NaN,1
290,Infracción ley n° 23.737 (estupefacientes),Robo y/o tentativa de robo,NaN,NaN,NaN,2
291,Robo y/o tentativa de robo,Amenazas,NaN,NaN,NaN,1


3- Generar dataframe con las variables de tipo "delito_xx" y la feature seleccionada en el paso anterior. Dividir los datos en conjunto de training y test.

In [47]:
df_delitos = df1.copy()
df_delitos[delito_columns] = df_delitos[delito_columns].astype('category')
cat_columns = df_delitos.select_dtypes(['category']).columns
df_delitos[cat_columns] = df_delitos[cat_columns].apply(lambda x: x.cat.codes)
df_delitos.head()

X = df_delitos['delito1']
y = df_delitos['duracion_condena_rango']

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)

4- Generar un modelo baseline, que devuelva el valor más frecuente de rango de condena para el crimen cometido (usaremos solo delito1_id). Calcular 3 métricas de evaluación de regresiones para este modelo.

In [48]:
def classify_condena_naive(X_train, X_test, y_train):
    
    y_pred = np.zeros(X_test.shape[0])
    for i, x in enumerate(X_test):
        mask = X_train == x
        y_pred[i] = np.bincount(y_train[mask]).argmax()
        
    return y_pred

def compute_metrics_classification(y_true, y_pred):
    accuracy = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred, average='weighted')
    recall = recall_score(y_true, y_pred, average='weighted')
    f1 = f1_score(y_true, y_pred, average='weighted')
    return accuracy, precision, recall, f1

y_pred = classify_condena_naive(X_train, X_test, y_train)
compute_metrics_classification(y_pred, y_test)

/home/nahuel/anaconda3/envs/diploDatos/lib/python3.7/site-packages/sklearn/metrics/classification.py:1145: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
/home/nahuel/anaconda3/envs/diploDatos/lib/python3.7/site-packages/sklearn/metrics/classification.py:1145: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


(0.3765659543109801,
 0.6363998402146075,
 0.3765659543109801,
 0.4589630082246017)

5- Entrenar una regresión logística y un árbol de decisión usando solo las variables de tipo "delitoxxx". Evaluar métricas para el conjunto de training y para test.

In [49]:
X = df_delitos[delito_columns]
y = df_delitos['duracion_condena_rango']

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)

model = DecisionTreeClassifier()
model.fit(X_train, y_train)

y_pred = model.predict(X_train)
print('Train:', compute_metrics_classification(y_pred, y_train))

y_pred = model.predict(X_test)
print('Test: ', compute_metrics_classification(y_pred, y_test))

Train: (0.41755711127487105, 0.5776760352425012, 0.41755711127487105, 0.46483023656498945)
Test:  (0.3858695652173913, 0.5445975659248679, 0.3858695652173913, 0.4377019296342151)


6- Seleccionar 3 variables adicionales para entrenar los modelos. Generar nuevamente conjuntos de training y test. Entrenar y evaluar los modelos de regresión lineal y árbol de decisión.

### Ajuste de Hiperparámetros

1- Elegir un modelo (puede ser de regresión o de clasificación) y realizar ajuste de hiperparámetros. Usar grid search y cross-validation. Reportar resultados.

In [50]:
for idx, criterion in enumerate((['mse']), start=1):
    exploring_params = {
        #'learning_rate': ['constant'],
        'splitter': ['best', 'random'],
        'max_depth': [1, 2, 3, 4, None],  
        'min_samples_leaf': [1, 10, 20, 50, 100] 
    }
    ad = DecisionTreeRegressor(criterion=criterion, random_state=0)
    model = GridSearchCV(ad, exploring_params, cv=5, scoring='neg_mean_squared_error')
    model.fit(X_train, y_train)
    
    print("# Exploración de hiperparámetros para criterion \"%s\"" % criterion, end="\n\n")
    
    print("Mejor conjunto de parámetros:")
    print(model.best_params_, end="\n\n")

    print("Puntajes de la grilla:", end="\n\n")
    means = model.cv_results_['mean_test_score']
    stds = model.cv_results_['std_test_score']
    for mean, std, params in zip(means, stds, model.cv_results_['params']):
        print("Exactitud: %0.4f (+/-%0.04f) para los parámetros %r" % (np.sqrt(-mean), np.sqrt(std), params))
    print()

    print("Reporte de clasificación para el mejor clasificador (sobre conjunto de evaluación):", end="\n\n")
    y_true, y_pred = y_test, model.predict(X_test)
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    print('RMSE = {:.4f}'.format(rmse), end='\n\n')
    print('Mejores parametros:')
    print(model.best_params_)
    print("================================================", end="\n\n")

# Exploración de hiperparámetros para criterion "mse"

Mejor conjunto de parámetros:
{'max_depth': None, 'min_samples_leaf': 10, 'splitter': 'best'}

Puntajes de la grilla:

Exactitud: 1.8451 (+/-0.2176) para los parámetros {'max_depth': 1, 'min_samples_leaf': 1, 'splitter': 'best'}
Exactitud: 1.8745 (+/-0.2296) para los parámetros {'max_depth': 1, 'min_samples_leaf': 1, 'splitter': 'random'}
Exactitud: 1.8451 (+/-0.2176) para los parámetros {'max_depth': 1, 'min_samples_leaf': 10, 'splitter': 'best'}
Exactitud: 1.8745 (+/-0.2296) para los parámetros {'max_depth': 1, 'min_samples_leaf': 10, 'splitter': 'random'}
Exactitud: 1.8451 (+/-0.2176) para los parámetros {'max_depth': 1, 'min_samples_leaf': 20, 'splitter': 'best'}
Exactitud: 1.8745 (+/-0.2296) para los parámetros {'max_depth': 1, 'min_samples_leaf': 20, 'splitter': 'random'}
Exactitud: 1.8451 (+/-0.2176) para los parámetros {'max_depth': 1, 'min_samples_leaf': 50, 'splitter': 'best'}
Exactitud: 1.8745 (+/-0.2296) para los paráme

2- Responder: ¿Mejoró sustancialmente el rendimiento de los modelos? ¿Por qué?

### OPCIONAL: Encoding de variables categóricas

Si tienen tiempo, lean el siguiente link: https://www.datacamp.com/community/tutorials/categorical-data.
Y luego ejecuten los siguientes puntos:


1- Usar la función get_dummies() de Pandas para obtener one-hot encoding de las variables categóricas

2- Seleccionar un modelo de regresión o clasificación y evaluar los resultados con las nuevas variables